In [ ]:
!sudo apt-get install ninja-build
!ninja --version

In [ ]:
import os

if os.path.exists('/content/nvdiffrast'):
  %rm -rf /content/nvdiffrast

!git clone --recursive https://github.com/NVlabs/nvdiffrast
%cd /content/nvdiffrast
!pip install .
%cd /content/

In [ ]:
import cv2

import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

import nvdiffrast.torch as dr

from tqdm import tqdm
from IPython.display import Image
from torchvision.transforms import ToPILImage, ToTensor
from einops import rearrange, reduce, repeat

In [ ]:
use_opengl = False # On T4 GPU, only False works, but rasterizer works much better if = True
glctx = dr.RasterizeGLContext() if use_opengl else dr.RasterizeCudaContext()